### Create AE

In [ ]:
# from keras.models import Model
# from keras.layers import Input
# from ae_model import encoder_vgg16, decoder1_vgg16, decoder2_vgg16, decoder3_vgg16, encoder1_vgg16

# input_shape = Input(shape=[256, 192, 3])
# ae = Model(input_shape, decoder1_vgg16(encoder_vgg16(input_shape)))
# ae.summary()

### Create Datasets

In [1]:
from ctypes import resize
import os
import cv2
import numpy as np
from tqdm import tqdm
from keras.preprocessing.image import img_to_array, load_img

datasets = []
path = './grid-search/dataset-bc/JPEGImages/'

files = os.listdir(path)


for i in tqdm(files):
    img = load_img(path+i, target_size=(192, 256))
    datasets.append(img_to_array(img))

data_array = np.reshape(datasets, (len(datasets), 256, 192, 3))
data_array = data_array.astype('float32') / 255.

data_array_2d = np.reshape(data_array, (data_array.shape[0], -1)) #sklearn need 2d array

print(img.size)
print(data_array.shape)
print(data_array_2d.shape)

100%|██████████| 366/366 [00:00<00:00, 935.69it/s]


(256, 192)
(366, 256, 192, 3)
(366, 147456)


In [2]:
from sklearn.model_selection import train_test_split

train_data,val_data = train_test_split(data_array,train_size=0.8)


In [ ]:
# from keras.callbacks import TensorBoard

# logdir = './grid-search/logs/test1/'

# ae.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# ae.fit(data_array, data_array, epochs=50,batch_size=40, shuffle=True, verbose=1, callbacks=[TensorBoard(log_dir=logdir)])


### GridSearch Test1    
Batch Size and Number of Epochs   
Fixed mse and acc

In [7]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input
from ae_model import encoder_vgg16, decoder1_vgg16, decoder2_vgg16, decoder3_vgg16, encoder1_vgg16
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor

def create_model():

    input_shape = Input(shape=[256, 192, 3])
    ae = Model(input_shape, decoder1_vgg16(encoder_vgg16(input_shape)))

    ae.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return ae

tf.random.set_seed(7)

grid_model1 = KerasRegressor(model=create_model)

batch_sizes = [8, 16, 32] # GPU memory Limit
epochs = [50, 100]

param_grid1 = dict(batch_size = batch_sizes, epochs = epochs)

grid1 = GridSearchCV(cv=[(slice(None), slice(None))], estimator=grid_model1, param_grid=param_grid1)

grid1.fit(data_array, data_array)

ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py", line 918, in _fit
    X, y = self._initialize(X, y)
  File "/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py", line 846, in _initialize
    X, y = self._validate_data(X, y, reset=True)
  File "/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py", line 1726, in _validate_data
    return super